In [1]:
from enum import auto
from random import random
from time import sleep, time
from tkinter import W
from turtle import xcor
import re
import random
from googleapiclient.discovery import build
from google.oauth2 import service_account

SERVICE_ACCOUNT_FILE = '/Users/sg2025/ScoreRanker/credentials.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

creds = None
creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# The ID spreadsheet.
SAMPLE_SPREADSHEET_ID = '19j2uvhSIJwTYByazsJRwGfzLrKoZjg3sN3UR2ZUUGO8'

service = build('sheets', 'v4', credentials=creds)



def get_team_list():
    #raw_list_of_teams is the raw data of the list of teams
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                        range="SVR2023!D2:D1305").execute()
    raw_list_of_teams = result.get('values', [])

    #list_of_teams_no_repetition is the raw data list of teams without repetition
    list_of_teams_no_repetition = []
    for teams in raw_list_of_teams:
        if teams not in list_of_teams_no_repetition:
            list_of_teams_no_repetition.append(teams)

    #final_list_of_teams is a list of numbers sorted in order from smallest to largest
    team_index = 0
    global final_list_of_teams 
    final_list_of_teams = []
    while team_index<(len(list_of_teams_no_repetition)):
        team_number =((list_of_teams_no_repetition[team_index][0]))
        team_number =int(team_number)
        final_list_of_teams.append(team_number)
        team_index=team_index+1

    final_list_of_teams.sort()
    print(final_list_of_teams)
    #Make sure to check for mistaken team number entries from this list'''

#final_list_of_teams = [100, 199, 253, 254, 581, 599, 604, 751, 766, 840, 841, 846, 852, 972, 1160, 1351, 1678, 1868, 1967, 2035, 2367, 2473, 2489, 2643, 2854, 3256, 3257, 3482, 3669, 4010, 4171, 4186, 4255, 4669, 4904, 4973, 4990, 5027, 5171, 5458, 5507, 5985, 6238, 6390, 6418, 6814, 6884, 7419, 7667, 7709, 8262, 8521, 8805, 8840, 8852, 9006, 9126, 9143]
#final_list_of_teams = [100, 199, 253, 254, 581, 599, 604, 649, 751, 766, 840, 841, 846, 852, 972, 1160, 1351, 1678, 1868, 2473, 2643, 2854, 3257, 3669, 4255, 5171, 5458, 5507, 5985, 6238, 7419, 7709, 8840, 8852]
final_list_of_teams = [2473, 4010, 3669]

teams_scores = []

unranked_team_list = []
unranked_auto_team_list = []
unranked_teleop_team_list = []

ranked_teams_scores = []
ranked_teams_autos = []
ranked_teams_teleops = []

scores = []
teams_autos = []
teams_teleops = []

global total_team_auto_score
global total_team_teleop_score
global total_team_endgame_score
global team_auto_avg
global team_teleop_avg
global total_team_average

def team_rank_calculation():
    index_of_team = 0
    team_number = 0
    total_team_average = 0
    
    #Goes through all teams list
    while len(final_list_of_teams)>index_of_team: 
        total_team_auto_score = 0
        total_team_teleop_score = 0
        total_team_endgame_score = 0
        team_auto_avg = 0
        team_teleop_avg = 0
        team_number = ((final_list_of_teams[index_of_team]))
        print("")
        print(team_number)
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                        range=str(team_number)+"!B3:B").execute()
        quals_scouted = result.get('values', [])
        num_quals_scouted = len(quals_scouted)
        
        qual_index=3
        #Goes through all quals per team
        while (qual_index<=num_quals_scouted+2):
            total_auto = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                        range=str(team_number)+"!AR" + str(qual_index) + ":AR" + str(qual_index)).execute()
            total_auto_value = total_auto.get('values',[])
            sleep(1)
            if (total_auto_value != []):
                total_auto_value_string = total_auto_value[0][0]
                auto_row_values_list = total_auto_value_string.split(", ")
                
                auto_row_index = 0
                auto_top_row_counter = 0
                auto_middle_row_counter = 0
                auto_bottom_row_counter = 0
                auto_engage_counter = 0
                auto_dock_counter = 0
                auto_none_counter = 0
                auto_community_zone = 0
                
                #Goes through all auto data per match
                while (auto_row_index<len(auto_row_values_list)):
                    row = auto_row_values_list[auto_row_index]
                    if (row == 'Top Row'):
                        auto_top_row_counter=auto_top_row_counter+1
                    elif (row == 'Middle Row'):
                        auto_middle_row_counter=auto_middle_row_counter+1
                    elif (row == 'Bottom Row'):
                        auto_bottom_row_counter=auto_bottom_row_counter+1
                    elif (row == 'Engage'):
                        auto_engage_counter = auto_engage_counter+1
                    elif (row == 'Dock'):
                        auto_dock_counter = auto_dock_counter+1
                    elif (row == 'None' or row == ''):
                        auto_none_counter = auto_none_counter + 1
                    elif (row == 'Yes'):
                        auto_community_zone = auto_community_zone+1
                        
                    auto_row_index=auto_row_index+1
                    
                global total_match_auto_score 
                total_match_auto_score = auto_top_row_counter*6 + auto_middle_row_counter*4 + auto_bottom_row_counter*3 + auto_engage_counter*12 + auto_dock_counter*8 + auto_community_zone*3 + auto_none_counter*0
                
                
            sleep(1)
                
            total_teleop = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                        range=str(team_number)+"!AS" + str(qual_index) + ":AS" + str(qual_index)).execute()
            total_teleop_value = total_teleop.get('values',[])
            if (total_teleop_value != []):
                total_teleop_value_string = total_teleop_value[0][0]
                teleop_row_values_list = total_teleop_value_string.split(", ")
                
                teleop_row_index = 0
                teleop_top_row_counter = 0
                teleop_middle_row_counter = 0
                teleop_bottom_row_counter = 0
                
                #Goes through all auto data per match
                while (teleop_row_index<len(teleop_row_values_list)):
                    row = teleop_row_values_list[teleop_row_index]
                    if (row == 'Top Row'):
                        teleop_top_row_counter=teleop_top_row_counter+1
                    elif (row == 'Middle Row'):
                        teleop_middle_row_counter=teleop_middle_row_counter+1
                    elif (row == 'Bottom Row'):
                        teleop_bottom_row_counter=teleop_bottom_row_counter+1
                        
                        
                    teleop_row_index=teleop_row_index+1
                    
                global total_match_teleop_score
                total_match_teleop_score = teleop_top_row_counter*5 + teleop_middle_row_counter*3 + teleop_bottom_row_counter*2
            
            
            
            
            sleep(1)

            
            total_endgame = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                        range=str(team_number)+"!AH" + str(qual_index) + ":AH" + str(qual_index)).execute()
            total_endgame_value = total_endgame.get('values',[])
            if (total_endgame_value != []):
                total_endgame_value_string = total_endgame_value[0][0]
                endgame_row_values_list = total_endgame_value_string.split(", ")
                
                endgame_row_index = 0
                endgame_engage_counter = 0
                endgame_dock_counter = 0
                endgame_park_counter = 0
                endgame_none_counter = 0
                
                #Goes through all auto data per match
                while (endgame_row_index<len(endgame_row_values_list)):
                    row = endgame_row_values_list[endgame_row_index]
                    if (row == 'Engage'):
                        endgame_engage_counter=endgame_engage_counter+1
                    elif (row == 'Dock'):
                        endgame_dock_counter=endgame_dock_counter+1
                    elif (row == 'Park'):
                        endgame_park_counter=endgame_park_counter+1
                    elif (row == '' or row == 'None'):
                        endgame_none_counter = endgame_none_counter+1
                        
                    endgame_row_index=endgame_row_index+1
                    
                global total_match_endgame_score
                total_match_endgame_score = endgame_engage_counter*10 + endgame_dock_counter*6 + endgame_park_counter*2 + endgame_none_counter*0
            
            
            total_team_auto_score = total_team_auto_score + total_match_auto_score
            total_team_teleop_score = total_team_teleop_score+total_match_teleop_score
            total_team_endgame_score = total_team_endgame_score + total_match_endgame_score            
            qual_index=qual_index+1
            
            sleep(1)

            
        team_auto_avg = total_team_auto_score/num_quals_scouted
        print("Auto: " + str(team_auto_avg))
        team_teleop_avg = total_team_teleop_score/num_quals_scouted
        print("Teleop: " + str(team_teleop_avg))
        team_endgame_avg = total_team_endgame_score/num_quals_scouted
        print("Endgame: " + str(team_endgame_avg))
        
        total_team_average = team_auto_avg+team_teleop_avg+team_endgame_avg
        print("Total: " + str(total_team_average))
        
        scores.append(total_team_average)
        teams_autos.append(team_auto_avg)
        teams_teleops.append(team_teleop_avg)
        
        unranked_team_list.append(team_number)
        unranked_team_list.append(total_team_average)
        
        unranked_auto_team_list.append(team_number)
        unranked_auto_team_list.append(team_auto_avg)
        
        unranked_teleop_team_list.append(team_number)
        unranked_teleop_team_list.append(team_teleop_avg)
        
        
        
        index_of_team=index_of_team+1
    
          
    scores.sort()
    scores.reverse()
    
    print(scores)
    print(unranked_team_list)
    print()
    
    scores_index=0
    while(scores_index<len(scores)):
        score_of_team = scores[scores_index]
        index = unranked_team_list.index(score_of_team)
        index = index - 1
        team_num = unranked_team_list[index]
        individual = "[" + "'" + str(team_num) + "' " + ", " + str(score_of_team) + "]"
        ranked_teams_scores.append(individual)
        scores_index=scores_index+1
        
    print("Total Score")
    print(ranked_teams_scores)
    
    
    teams_autos.sort()
    teams_autos.reverse()
    
    print(teams_autos)
    print(unranked_auto_team_list)
    print()
    
    auto_scores_index=0
    while(auto_scores_index<len(scores)):
        score_of_team = teams_autos[auto_scores_index]
        index = unranked_auto_team_list.index(score_of_team)
        index = index - 1
        team_num = unranked_auto_team_list[index]
        individual = "[" + "'" + str(team_num) + "' " + ", " + str(score_of_team) + "]"
        ranked_teams_autos.append(individual)
        auto_scores_index=auto_scores_index+1
    print("Auto Score")
    print(ranked_teams_autos)
    
    teams_teleops.sort()
    teams_teleops.reverse()
    
    print(teams_teleops)
    print(unranked_teleop_team_list)
    print()
    
    teleop_scores_index=0
    while(teleop_scores_index<len(scores)):
        score_of_team = teams_teleops[teleop_scores_index]
        index = unranked_teleop_team_list.index(score_of_team)
        index = index - 1
        team_num = unranked_teleop_team_list[index]
        individual = "[" + "'" + str(team_num) + "' " + ", " + str(score_of_team) + "]"
        ranked_teams_teleops.append(individual)
        teleop_scores_index=teleop_scores_index+1
    print("Teleop Score")
    print(ranked_teams_teleops)
    

        
        
        
        

#get_team_list()
team_rank_calculation()




2473


timeout: The read operation timed out

In [51]:
#teleop = [['1678' , 35.875], ['254' , 35.625], ['604' , 31.666666666666668], ['581' , 30.88888888888889], ['1351' , 27.0], ['649' , 25.0], ['972' , 24.666666666666668], ['4255' , 23.833333333333332], ['599' , 23.375], ['3669' , 22.0], ['1868' , 21.75], ['846' , 21.375], ['3257' , 18.857142857142858], ['100' , 18.0], ['1160' , 16.88888888888889], ['5507' , 14.777777777777779], ['8840' , 14.666666666666666], ['2854' , 14.0], ['199' , 12.333333333333334], ['5458' , 12.272727272727273], ['840' , 11.0], ['5171' , 10.0], ['5171' , 10.0], ['766' , 9.571428571428571], ['8852' , 9.222222222222221], ['2473' , 7.714285714285714], ['6238' , 7.666666666666667], ['5985' , 7.25], ['852' , 7.2], ['751' , 5.2], ['253' , 4.666666666666667], ['2643' , 4.444444444444445], ['7709' , 4.333333333333333]]
teleop = [['254' , 35.1], ['254' , 35.1], ['581' , 30.88888888888889], ['604' , 28.857142857142858], ['1351' , 26.88888888888889], ['4255' , 26.5], ['599' , 23.555555555555557], ['846' , 22.7], ['649' , 22.285714285714285], ['1868' , 22.166666666666668], ['972' , 22.142857142857142], ['3257' , 20.5], ['3669' , 18.0], ['1160' , 17.90909090909091], ['5507' , 16.75], ['100' , 15.333333333333334], ['2854' , 13.444444444444445], ['8840' , 12.909090909090908], ['841' , 12.857142857142858], ['199' , 12.428571428571429], ['5458' , 12.272727272727273], ['5171' , 10.875], ['840' , 10.666666666666666], ['8852' , 9.454545454545455], ['766' , 9.181818181818182], ['7419' , 9.090909090909092], ['2473' , 8.625], ['5985' , 8.222222222222221], ['6238' , 7.5], ['852' , 6.285714285714286], ['751' , 6.25], ['253' , 5.083333333333333], ['7709' , 4.25], ['2643' , 3.6923076923076925]]
sheet = service.spreadsheets()
'''result = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID, 
                            range="Rankings!A2:B", valueInputOption="USER_ENTERED", body={"values":average}).execute()

result = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID, 
                            range="Rankings!D2:E", valueInputOption="USER_ENTERED", body={"values":auto}).execute()'''

result = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID, 
                            range="Rankings!G2:H", valueInputOption="USER_ENTERED", body={"values":teleop}).execute()
